In [5]:
#Importación de los modulos necesarios y lectura del archivo CSV y Reemplazar los valores NaN en la columna de 'harassmentRisk'
from geopy.geocoders import Nominatim
from queue import PriorityQueue
import pandas as pd
import folium

df = pd.read_csv(r'calles_de_medellin_con_acoso.csv', sep=";")
HARASSMENT_APROX = df['harassmentRisk'].mean()
df['harassmentRisk'].fillna(HARASSMENT_APROX, inplace=True)

In [6]:
#Metodo para corregir las coordenadas
def correctFormat(text: str) -> tuple:
    textProcess = text[1:-1]
    partB, partA = textProcess.split(",")
    listReturn = [float(partA),float(partB)]
    return tuple(listReturn)

#Metodo para obtener el nombre de una calle con sus coordenadas
def coordinateToStreet(coordinate: tuple) -> str:
    localizator = Nominatim(user_agent = "SecurityON")
    location = localizator.reverse("{},{}".format(coordinate[0],coordinate[1]))
    return location

In [12]:
#Metodo que crea el grafo
def createGrahp() -> dict:
    uniqueOrigins = pd.unique(df['origin'])
    graph = {}
    for origin in uniqueOrigins:
        correctOrigin = correctFormat(origin)
        graph[correctOrigin] = {}

    for index in df.index:
        originIteration = correctFormat(df['origin'][index])
        destinationIteration = correctFormat(df['destination'][index])
        if destinationIteration in graph:
            if df['oneway'][index]:
                graph[originIteration] |= {destinationIteration:(df['length'][index],df['harassmentRisk'][index])}
                graph[destinationIteration] |= {originIteration:(df['length'][index],df['harassmentRisk'][index])}
            else: graph[originIteration] |= {destinationIteration:(df['length'][index],df['harassmentRisk'][index])}
        else:
            if df['oneway'][index]:
                graph[destinationIteration] = {}
                graph[originIteration] |= {destinationIteration:(df['length'][index],df['harassmentRisk'][index])}
                graph[destinationIteration] |= {originIteration:(df['length'][index],df['harassmentRisk'][index])}
            else: graph[originIteration] |= {destinationIteration:(df['length'][index],df['harassmentRisk'][index])}

    return graph

graphCM = createGrahp()

In [14]:
def create_map(paht: list):
    map = folium.Map(location=[paht[0][0], paht[0][1]], zoom_start=14)
    folium.PolyLine(paht, color='blue', weigth=15, opacity=0.8).add_to(map)
    map.save('index.html')

def DjikstraSecond(graph: dict, origin: tuple, destination: tuple) -> list:
    distances = {vertex : float("inf") for vertex in graph}
    previus = {vertex: 0 for vertex in graph}
    compares = PriorityQueue()

    distances[origin] = 0
    compares.put((0,origin))

    while not compares.empty():
        vertexTuple = compares.get()
        vertex = vertexTuple[1]
        if vertex == destination: break
        for adyacent in graph[vertex]:
            weigth = graph[vertex][adyacent][0]
            if distances[adyacent] > distances[vertex] + weigth:
                distances[adyacent] = distances[vertex] + weigth
                previus[adyacent] = vertex
                compares.put((distances[adyacent],adyacent))

    ruta = []
    actual = destination
    while actual != origin:
        ruta.insert(0,actual)
        actual = previus[actual]

    return ruta

ruta = DjikstraSecond(graphCM,(6.2099192, -75.5663399),(6.3003479,-75.5543665))

create_map(ruta)